# Week 3: Classification

In this section we will apply what we have learned about the logistic regression model to fit a model and make predictions. We will be using the `penguins` dataset from `seaborn` and try to predict where or not a penguin is of Adelie species.

We will build a Logistic Regression model from scratch and see how it compares to 


In [9]:
# Code for Week 5
import pandas as pd
import numpy as np
import seaborn as sns

# Import penguins
penguins = sns.load_dataset("penguins")
penguins["Adelie"] = 1*(penguins["species"] == "Adelie")

print("Data Shape, ",penguins.shape)

# Take a look at the columns
print(penguins.head())

# What percentage of our data is Adelie
print(np.mean(penguins.Adelie))

# What features do we have
print(penguins.columns)








Data Shape,  (344, 8)
  species     island  bill_length_mm  ...  body_mass_g     sex  Adelie
0  Adelie  Torgersen            39.1  ...       3750.0    MALE       1
1  Adelie  Torgersen            39.5  ...       3800.0  FEMALE       1
2  Adelie  Torgersen            40.3  ...       3250.0  FEMALE       1
3  Adelie  Torgersen             NaN  ...          NaN     NaN       1
4  Adelie  Torgersen            36.7  ...       3450.0  FEMALE       1

[5 rows x 8 columns]
0.4418604651162791
Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'Adelie'],
      dtype='object')
